In [42]:
from pptx.util import Inches, Pt, Cm
from pptx.chart.data import CategoryChartData
from pptx.enum.chart import XL_CHART_TYPE
from pptx.enum.text import MSO_AUTO_SIZE
from pptx import Presentation
import pandas as pd

In [147]:
class pandas_2_pptx():

    def __init__(self, fname=None):
        '''
        pandasのdataframeをpptxのスライドとして出力するクラス

        Args:
            fname (string): pptxファイルのパス。指定した場合、
            読み込んだpptxにデータが追記される。指定しない場合新しいファイルに出力される。
        '''
        if(fname is None):
            self.prs = Presentation()
        else:
            self.prs = Presentation(fname)
            print(f"{fname}を読み込みました。\n上書きに気をつけてください")

    def add_title_slide(self, title_txt):
        '''
        タイトルスライドを追加する

        Args:
            title_txt (string): スライドのタイトル名

        return:
            slide (slide): slideオブジェクト
        '''
        slide_obj = self.prs.slides.add_slide(self.prs.slide_layouts[0])
        title_slide = slide_obj.shapes.title
        title_slide.text = title_txt
        slide_obj = slide(slide_obj)
        return slide_obj

    def add_slide(self, title_txt):
        '''
        スライドを追加する

        Args:
            title_txt (string): スライドのタイトル名

        return:
            slide (slide): slideオブジェクト
        '''
        slide_obj = self.prs.slides.add_slide(self.prs.slide_layouts[1])
        title = slide_obj.shapes.title
        title.text = title_txt
        slide_obj = slide(slide_obj)
        return slide_obj

    def save(self, fname):
        '''
        スライドを追加したpptxデータを出力する

        Args:
            fname (string): 出力するpptxファイルのパス。
        '''
        self.prs.save(fname)
     
        
class slide():

    def __init__(self, slide_obj):
        '''
        pptxクラスのslideオブジェクトにデータを追加するクラス

        Args:
            slide (pptx.slide.Slide): pptxクラスのslideオブジェクト。
        '''
        self.__slide = slide_obj
        self.__slide_object_list = []
        self.__df_list = []

    def add_chart(self, df):
        '''
        プライベートメソッド。
        pandasをpptxの棒グラフに変換する関数

        Args:
            df (pandas.DataFrame): 変換するdataframe
        '''
        self.__slide_object_check()
        chart_data = self.__pandas_genelate_linechart(df)
        self.__df_list.append(df)
        self.__slide_object_list.append(["chart", chart_data])

    def add_table(self, df):
        '''
        プライベートメソッド。
        pandasをpptxのテーブルに変換する関数

        Args:
            df (pandas.DataFrame): 変換するdataframe
        '''
        for i in df.select_dtypes(float).columns:
            df[i] = df[i].round(2)
        self.__slide_object_check()
        rows = len(df)+1
        cols = len(df.columns)+1
        self.__df_list.append(df)
        self.__slide_object_list.append(["table", rows, cols])

    def __pandas_genelate_table(self, table, df):
        '''
        プライベートメソッド。
        pandasをpptxのテーブルに変換する関数

        Args:
            table (pptx.table.Table): tableオブジェクト
            df (pandas.DataFrame): tableオブジェクトに格納するdataframe
        '''
        table.cell(0, 0).text = "index"
        text_frame = table.cell(0, 0).text_frame
        text_frame.word_wrap = True
        text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE
        for i, column in enumerate(df.columns):
            table.cell(0, i + 1).text = column
            text_frame = table.cell(0, i + 1).text_frame
            text_frame.word_wrap = True
            text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE

        for i, index in enumerate(df.index):
            table.cell(i + 1, 0).text = str(index)
            text_frame = table.cell(i + 1, 0).text_frame
            text_frame.word_wrap = True
            text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE

        for row in range(df.shape[0]):
            for line in range(df.shape[1]):
                table.cell(row+1, line+1).text = str(df.values[row, line])
                text_frame = table.cell(row+1, line+1).text_frame
                text_frame.word_wrap = True
                text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE

    def __pandas_genelate_linechart(self, df):
        '''
        プライベートメソッド。
        pandasのデータフレームをpptxのchartに変換

        Args:
            df (pandas.DataFrame): 出力したいdataframe

        return:
            chart_data (pptx.chart.data.CategoryChartData): chartデータ
        '''
        chart_data = CategoryChartData()
        chart_data.categories = df.index
        for column in df.columns:
            chart_data.add_series(column, df[column])
        return chart_data

    def __slide_object_check(self):
        '''
        プライベートメソッド。
        グラフやテーブルの数を6つ以上追加しようとするとエラー(それ以上は見栄えが悪くなる)
        '''
        if(len(self.__slide_object_list) == 6):
            raise Exception("objectが多すぎます")

    def __cordinate_shape(self):
        '''
        プライベートメソッド。
        グラフやテーブルの大きさを調整

        return:
            shape_list (list): グラフやテーブルの大きさを格納したリスト
        '''
        object_num = len(self.__slide_object_list)
        split_len = Cm(0.43)
        hight = Cm(14.71)
        width = Cm(24.4)
        left = Inches(0.17)
        top = Inches(1.5)
        shape_list = []

        if(object_num == 1):
            shape_list.append([left + split_len * 5,
                               top + split_len*4,
                               Cm(20.4),
                               Cm(11.5)])

        if(object_num <= 2):
            width = int((width - split_len) / 2)
            hight = int((hight - split_len*2) / 1)
            for i in range(object_num):
                shape_list.append([left + (width + split_len) * i, top, width, hight])

        elif(object_num <= 4):
            width = int((width - split_len) / 2)
            hight = int((hight - split_len*2) / 2)
            for i in range(2):
                for j in range(2):
                    shape_list.append([left + (width + split_len) * j, top + (hight + split_len) * i, width, hight])

        elif(object_num <= 6):
            width = int((width - split_len * 2) / 3)
            hight = int((hight - split_len*2) / 2)
            for i in range(2):
                for j in range(3):
                    shape_list.append([left + (width + split_len) * j, top + (hight + split_len) * i, width, hight])
        return shape_list

    def generate(self):
        '''
        プライベートメソッド。
        追加したグラフやテーブルを__cordinate_shape()の情報をもとにスライドに書き込む
        '''
        shape_list = self.__cordinate_shape()
        for n, i in enumerate(self.__slide_object_list):
            if(i[0] == "table"):
                table = self.__slide.shapes.add_table(i[1],  # row
                                                      i[2],  # col
                                                      shape_list[n][0],  # left
                                                      shape_list[n][1],  # top
                                                      shape_list[n][2],  # width
                                                      shape_list[n][3]  # hight
                                                      ).table
                self.__pandas_genelate_table(table, self.__df_list[n])

            elif(i[0] == "chart"):
                chart = self.__slide.shapes.add_chart(XL_CHART_TYPE.LINE,
                                                      shape_list[n][0],  # left
                                                      shape_list[n][1],  # top
                                                      shape_list[n][2],  # width
                                                      shape_list[n][3],  # hight
                                                      i[1]
                                                      ).chart
                chart.has_legend = True
                chart.legend.include_in_layout = False


In [148]:
test_df = pd.DataFrame({"a":[1,2,3,4,5],"b":[-12,-23,-34,-45,-56],"c":[12,11,5,9,20]},index=[0,1,2,3,4])

In [149]:
for i in test_df.select_dtypes(float).columns:
    test_df[i] = test_df[i].round(2)
    print(i)

In [150]:
#読み込んがパワポのデータに追加もできる
#pd2pptx = pandas_2_pptx("test.pptx")
pd2pptx = pandas_2_pptx()

In [151]:
slide1 = pd2pptx.add_title_slide("pandasのdataframeを\nパワーポイントに自動変換")

In [152]:
slide2 = pd2pptx.add_slide("テーブルを一つ出力")
slide2.add_table(test_df)
slide2.generate()

In [153]:
slide3 = pd2pptx.add_slide("グラフを一つ出力")
slide3.add_chart(test_df)
slide3.generate()

In [154]:
slide4 = pd2pptx.add_slide("テーブルとグラフを出力")
slide4.add_chart(test_df)
slide4.add_table(test_df)
slide4.generate()

In [155]:
slide5 = pd2pptx.add_slide("いろいろ4つ出力")
slide5.add_chart(test_df)
slide5.add_table(test_df)
slide5.add_table(test_df)
slide5.add_table(test_df)
slide5.generate()

In [156]:
slide6 = pd2pptx.add_slide("いろいろ5つ出力")
slide6.add_chart(test_df)
slide6.add_chart(test_df)
slide6.add_chart(test_df)
slide6.add_table(test_df)
slide6.add_table(test_df)
slide6.generate()

In [157]:
slide7 = pd2pptx.add_slide("7つは出力できない")
slide7.add_chart(test_df)
slide7.add_chart(test_df)
slide7.add_chart(test_df)
slide7.add_table(test_df)
slide7.add_table(test_df)
slide7.add_table(test_df)
slide7.add_table(test_df)
slide7.generate()

Exception: objectが多すぎます

In [158]:
slide7.generate()

In [159]:
pd2pptx.save("test2.pptx")